In [ ]:
# app
from vois.vuetify import app
a = app.app(totalheight=400)
a.show()

In [ ]:
# button
from vois.vuetify import button
b = button.button('Test button', selected=True)
display(b.draw())

In [ ]:
# dialogYesNo
from ipywidgets import widgets, Layout
from IPython.display import display
from vois.vuetify import dialogYesNo

output = widgets.Output(layout=Layout(height='100px'))
display(output)

def on_yes():
    with output:
        print('YES')

def on_no():
    with output:
        print('NO')
    
dlg = dialogYesNo.dialogYesNo(title='Question', text='Confirm removal of the selected file?', titleheight=40, width=400, output=output,
                              show=True, transition='dialog-bottom-transition', on_yes=on_yes, on_no=on_no)

In [ ]:
# treeview
from vois.vuetify import treeview
treecard = treeview.createTreeviewFromList(['A','A.1','A.2','A.1.1','A.3.1','A.4.1.2','A.5.2.1','A.4.2.3.1','B','B.A'], rootName='Root', expand_selection_to_parents=False, 
                                           substitutionDict={'A.1': 'A.1 new name'}, color='green', height=350)
treecard

In [ ]:
# Treemap/Sunburst/Icicle chart generated using treemapPlotly
import plotly.express as px
import plotly.graph_objects as go

from vois import treemapPlotly

valuefor = {'JRC.A.1': 3.0, 'JRC.A.2': 5.0, 'JRC.A.2.1': 4.0, 'JRC.B.1': 12.0, 'JRC.B.2': 7.0, 'JRC.B.3': 3.0, 'JRC.C.1': 7.0, 'JRC.C.2': 2.0}
labels, parents, values = treemapPlotly.createTreemapFromList(['JRC.A','JRC.A.1','JRC.A.2','JRC.A.2.1','JRC.B.1','JRC.B.2','JRC.B.3','JRC.C.1','JRC.C.2'],
                                                              rootName='JRC', valuefor=valuefor)

fig = go.Figure()
fig.add_trace(go.Treemap(ids=labels, labels=labels, parents=parents, values=values, branchvalues='total', maxdepth=4, root_color="lightgrey"))
#fig.add_trace(go.Sunburst(ids=labels, labels=labels, parents=parents, values=values, branchvalues='total', maxdepth=4, root_color="lightgrey"))
#fig.add_trace(go.Icicle(ids=labels, labels=labels, parents=parents, values=values, branchvalues='total', maxdepth=4, root_color="lightgrey"))
fig.update_layout(margin=dict(t=38, l=0, r=0, b=0), height=400, hoverlabel=dict(bgcolor="#eee", align="left"),
                  title={'text': "JRC units", 'y':1.0, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'})
fig.show()

In [ ]:
# title with sidePanel
from ipywidgets import widgets, Layout, HTML
from IPython.display import display
from vois.vuetify import title, switch, sidePanel

output = widgets.Output(layout=Layout(width='99%', height='400px'))
display(output)

def onclick(arg):
    with output:
        print(arg)

def onmenu():
    panel.show()
    
def on_s_change(value):
    with output:
        print(value)
    
s = switch.switch(True, 'PNG format', onchange=on_s_change)

panel = sidePanel.sidePanel(title='Settings for download', text='Please select the format for download:', 
                            width=400, right=False, content=[s.draw()], output=output, zindex=9999)

f = title.title(text='Energy consumption dashboard', 
                menu=True, onmenuclick=onmenu,
                buttons=['Home', 'About Us', 'Team', 'Services', 'Blog', 'Contact Us'],
                logo='https://jeodpp.jrc.ec.europa.eu/services/shared/home/images/JRCBigDataPlatform2.png', 
                logowidth=180, logomarginy=5, menumarginy=5,
                height=78, onclick=onclick, output=output)

In [ ]:
# intermap: load of a local geojson file, join of values to features, creation of a graduated legend and display of map and legend
from ipywidgets import widgets, Layout
from IPython.core.display import HTML as coreHTML
import numpy as np
import pandas as pd

from jeodpp import inter, imap
from vois import geojsonUtils, interMap

# Display a custom geojson file and create a graduated legend
country_codes = ['LV','AL','CH','AT','HR','ES','IS','RO','IT','VA','HU','SE','NL','CZ','DE','FR',
                 'ME','BE','SI','LU','IE','BA','MC','BG','PL','LI','GB','RS','SM','DK','IM',
                 'EE','SK','GR','LT','NO','PT','AD','MK','MT','FI','XK','CY','TR']

# Generate random values and create a dictionary: key=countrycode, value=random value in [0.0,100.0]
d = dict(zip(country_codes,list(np.random.uniform(size=len(country_codes),low=0.0,high=100.0))))

geojson = geojsonUtils.geojsonLoadFile('ne_50m_admin_0_countries.geojson')
geojsonnew = geojsonUtils.geojsonJoin(geojson,'ISO_A2_EH', 'value', d, innerMode=True)

vector = interMap.interGeojsonToVector(geojsonnew)

# Create a graduated legend
vector.colorScheme('invReds_soft')
vector.legendSet('line', 'stroke-width', '1.0')
vector.legendSet('line', 'stroke',       '#010101')
vector.legendGraduated("value","quantile",8)

vector.parameter("identifyseparator","<br>")
vector.parameter("identifyfield","FID NAME value")

# Add the VectorLayer to the map
m = imap.Map(basemap=1, layout=Layout(width='79%',height='600px'))
layer = m.addLayer(vector.toLayer())
extents = m.zoomToImageExtent(vector)

inter.identifyPopup(m,vector)

outlegend = widgets.Output(layout=Layout(width='20%',height='600px'))
with outlegend:
    display(coreHTML(vector.legend("Legend on value attribute", 1)))

widgets.HBox([m,outlegend])

In [ ]:
d